### Приложение 1

#### Установка PostgreSQL из исходного кода

Существует множество методов установки сервера PostgreSQL, но наиболее универсальным методом является установка из исходного кода. Этот метод, в случае необходимости, позволяет собрать сервер PostgreSQL с не стандартными параметрами или на специфической архитектуре. 

Загрузить исходный код последней стабильной версии PostgreSQL можно по ссылке https://ftp.postgresql.org/pub/source/v15.7/postgresql-15.7.tar.bz2

In [12]:
# Загрузим исходный код по этой ссылке в папку src 
# и назову файл postgresql.tar.bz2

!wget -O /var/lib/postgresql/postgresql.tar.bz2 https://ftp.postgresql.org/pub/source/v15.7/postgresql-15.7.tar.bz2

--2024-06-06 16:53:59--  https://ftp.postgresql.org/pub/source/v15.7/postgresql-15.7.tar.bz2
Resolving ftp.postgresql.org (ftp.postgresql.org)... 147.75.85.69, 87.238.57.227, 72.32.157.246, ...
Connecting to ftp.postgresql.org (ftp.postgresql.org)|147.75.85.69|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 23112318 (22M) [application/octet-stream]
Saving to: ‘src/postgresql.tar.bz2’

src/postgresql.tar. 100%[===================>]  22.04M  3.68MB/s    in 16s     

2024-06-06 16:54:19 (1.39 MB/s) - ‘src/postgresql.tar.bz2’ saved [23112318/23112318]



In [13]:
# Проверим существует ли загруженный файл

# !ls -l src/postgresql.tar.bz2

-rw-r--r-- 1 evgeny evgeny 23112318 May  7 04:36 src/postgresql.tar.bz2


In [14]:
# Проверим корректность загруженных пакетов сравнив хеш
# Хеш SHA256 доступен по ссылке 
# https://ftp.postgresql.org/pub/source/v16.3/postgresql-16.3.tar.bz2.sha256

%env PG_SHA256=a46fe49485ab6385e39dabbbb654f5d3049206f76cd695e224268729520998f7
!echo "$PG_SHA256 src/postgresql.tar.bz2" | sha256sum -c -

env: PG_SHA256=a46fe49485ab6385e39dabbbb654f5d3049206f76cd695e224268729520998f7
src/postgresql.tar.bz2: OK


In [16]:
# Разархивируем загруженный проект

!tar --extract --file src/postgresql.tar.bz2 --directory src --strip-components 1
!ls -l src

total 23356
-rw-r--r--  1 evgeny evgeny      397 May  7 04:23 aclocal.m4
drwxr-xr-x  2 evgeny evgeny     4096 May  7 04:34 config
-rwxr-xr-x  1 evgeny evgeny   602097 May  7 04:23 configure
-rw-r--r--  1 evgeny evgeny    89356 May  7 04:23 configure.ac
drwxr-xr-x 61 evgeny evgeny     4096 May  7 04:34 contrib
-rw-r--r--  1 evgeny evgeny     1192 May  7 04:23 COPYRIGHT
drwxr-xr-x  3 evgeny evgeny     4096 May  7 04:34 doc
-rw-r--r--  1 evgeny evgeny     4264 May  7 04:23 GNUmakefile.in
-rw-r--r--  1 evgeny evgeny      277 May  7 04:23 HISTORY
-rw-r--r--  1 evgeny evgeny    63842 May  7 04:35 INSTALL
-rw-r--r--  1 evgeny evgeny     1875 May  7 04:23 Makefile
-rw-r--r--  1 evgeny evgeny 23112318 May  7 04:36 postgresql.tar.bz2
-rw-r--r--  1 evgeny evgeny     1213 May  7 04:23 README
drwxr-xr-x 16 evgeny evgeny     4096 May  7 04:35 src


In [17]:
# Создам директорию для хранения файлов базы данных
# Создам каталог локал для сборки проекта
# Создам группу и суперпользователя postgres
# Сменю владельца data на пользователя postgres

# !mkdir -p data
# !mkdir -p .local
!id postgres
# !echo 314159 | sudo -S "chown postgres:postgres data"
!ls -la

uid=118(postgres) gid=127(postgres) groups=127(postgres),110(ssl-cert)
total 3140
drwxr-xr-x  6 evgeny evgeny    4096 Jun  6 16:56  .
drwxr-xr-x 19 evgeny evgeny    4096 Jun  6 16:07  ..
-rw-r--r--  1 evgeny evgeny   36120 Jun  6 16:50  config.log
drwxr-xr-x  2 evgeny evgeny    4096 Jun  6 16:13  data
-rw-r--r--  1 evgeny evgeny   13469 Jun  6 16:56  demo1.ipynb
drwxr-xr-x  2 evgeny evgeny    4096 Jun  6 16:50  .ipynb_checkpoints
drwxr-xr-x  2 evgeny evgeny    4096 Jun  6 16:33  .local
-rw-r--r--  1 evgeny evgeny   17389 Jun  6 10:53  sharaev_assignment.docx
-rw-r--r--  1 evgeny evgeny   40866 Jun  6 10:54  sharaev_assignment.pdf
-rw-r--r--  1 evgeny evgeny   15813 Jun  6 10:57  sharaev_feedback.docx
-rw-r--r--  1 evgeny evgeny   32616 Jun  6 16:00  sharaev_report.docx
drwxr-xr-x  6 evgeny evgeny    4096 Jun  6 16:56  src
-rw-r--r--  1 evgeny evgeny  802384 Jun  6 11:08  Исх.01_04-2024_01042024.pdf
-rw-r--r--  1 evgeny evgeny 2216340 Jun  5 18:22 'Шараев Е.В.pdf'


In [31]:
# Сконфигурирую новый сервер PostrreSQL
# включаем поддержку русского языка
# нам нужна перемещаемая инсталляция
# устанавливает номер порта 5432
# каталог для сборки сервера
# включаем поддержку Perl, Python
# включаем поддержку OPENSSL

!src/configure \
    --build="$gnuArch" \
    --enable-nls='ru' \
    --disable-rpath \
    --with-pgport=5432 \
    --prefix="/home/evgeny/Projects/internship/.local" \
    --with-python \
    --with-openssl

checking build system type... x86_64-pc-linux-gnu
checking host system type... x86_64-pc-linux-gnu
checking which template to use... linux
checking whether NLS is wanted... yes
checking for default port number... 5432
checking for block size... 8kB
checking for segment size... 1GB
checking for WAL block size... 8kB
checking for gcc... gcc
checking whether the C compiler works... yes
checking for C compiler default output file name... a.out
checking for suffix of executables... 
checking whether we are cross compiling... no
checking for suffix of object files... o
checking whether we are using the GNU C compiler... yes
checking whether gcc accepts -g... yes
checking for gcc option to accept ISO C89... none needed
checking for gcc option to accept ISO C99... none needed
checking for g++... g++
checking whether we are using the GNU C++ compiler... yes
checking whether g++ accepts -g... yes
checking for gawk... no
checking for mawk... mawk
checking whether gcc supports -Wdeclaration-after-

checking langinfo.h usability... yes
checking langinfo.h presence... yes
checking for langinfo.h... yes
checking mbarrier.h usability... no
checking mbarrier.h presence... no
checking for mbarrier.h... no
checking poll.h usability... yes
checking poll.h presence... yes
checking for poll.h... yes
checking sys/epoll.h usability... yes
checking sys/epoll.h presence... yes
checking for sys/epoll.h... yes
checking sys/event.h usability... no
checking sys/event.h presence... no
checking for sys/event.h... no
checking sys/ipc.h usability... yes
checking sys/ipc.h presence... yes
checking for sys/ipc.h... yes
checking sys/personality.h usability... yes
checking sys/personality.h presence... yes
checking for sys/personality.h... yes
checking sys/prctl.h usability... yes
checking sys/prctl.h presence... yes
checking for sys/prctl.h... yes
checking sys/procctl.h usability... no
checking sys/procctl.h presence... no
checking for sys/procctl.h... no
checking sys/pstat.h usability... no
checking sys

checking alignment of PG_INT128_TYPE... 16
checking for builtin __sync char locking functions... yes
checking for builtin __sync int32 locking functions... yes
checking for builtin __sync int32 atomic operations... yes
checking for builtin __sync int64 atomic operations... yes
checking for builtin __atomic int32 atomic operations... yes
checking for builtin __atomic int64 atomic operations... yes
checking for __get_cpuid... yes
checking for __cpuid... no
checking for _mm_crc32_u8 and _mm_crc32_u32 with CFLAGS=... no
checking for _mm_crc32_u8 and _mm_crc32_u32 with CFLAGS=-msse4.2... yes
checking for __crc32cb, __crc32ch, __crc32cw, and __crc32cd with CFLAGS=... no
checking for __crc32cb, __crc32ch, __crc32cw, and __crc32cd with CFLAGS=-march=armv8-a+crc... no
checking which CRC-32C implementation to use... SSE 4.2 with runtime check
checking for library containing sem_init... none required
checking which semaphore API to use... unnamed POSIX
checking which random number source to use..

Этот скрипт провел несколько проверок с целью определить значения для различных переменных, зависящих от системы, а затем создал несколько файлов в дереве сборки, в которых отразил полученные результаты.

In [42]:
!ls -l config*

-rw-r--r-- 1 evgeny evgeny 492970 Jun  6 17:19 config.log
-rwxr-xr-x 1 evgeny evgeny  41252 Jun  6 17:19 config.status

config:
total 0
lrwxrwxrwx 1 evgeny evgeny 52 Jun  6 17:19 Makefile -> /home/evgeny/Projects/internship/src/config/Makefile


In [45]:
# После того как конфигурация завершилась можно
# приступить к сборке сервера

!make world

make -C ./src/backend generated-headers
make[1]: Entering directory '/home/evgeny/Projects/internship/src/backend'
make -C catalog distprep generated-header-symlinks
make[2]: Entering directory '/home/evgeny/Projects/internship/src/backend/catalog'
make[2]: Nothing to be done for 'distprep'.
make[2]: Nothing to be done for 'generated-header-symlinks'.
make[2]: Leaving directory '/home/evgeny/Projects/internship/src/backend/catalog'
make -C utils distprep generated-header-symlinks
make[2]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils'
make[2]: Nothing to be done for 'distprep'.
make[2]: Nothing to be done for 'generated-header-symlinks'.
make[2]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils'
make[1]: Leaving directory '/home/evgeny/Projects/internship/src/backend'
make -C doc all
make[1]: Entering directory '/home/evgeny/Projects/internship/doc'
make -C src all
make[2]: Entering directory '/home/evgeny/Projects/internship/doc/src'
make 

make[4]: Entering directory '/home/evgeny/Projects/internship/src/backend/replication/logical'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/backend/replication/logical'
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/replication'
make -C rewrite all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/rewrite'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/rewrite'
make -C statistics all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/statistics'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/statistics'
make -C storage all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/storage'
make -C buffer all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/backend/storage/buffer'
make[4]: Nothing 

make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils/mb/conversion_procs/euc_jp_and_sjis'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils/mb/conversion_procs/euc_jp_and_sjis'
make -C euc_kr_and_mic all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils/mb/conversion_procs/euc_kr_and_mic'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils/mb/conversion_procs/euc_kr_and_mic'
make -C euc_tw_and_big5 all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils/mb/conversion_procs/euc_tw_and_big5'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils/mb/conversion_procs/euc_tw_and_big5'
make -C latin2_and_win1250 all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils/mb/conversion_procs/

make[5]: Entering directory '/home/evgeny/Projects/internship/src/interfaces/libpq'
make -C ../../../src/port all
make[6]: Entering directory '/home/evgeny/Projects/internship/src/port'
make[6]: Nothing to be done for 'all'.
make[6]: Leaving directory '/home/evgeny/Projects/internship/src/port'
make -C ../../../src/common all
make[6]: Entering directory '/home/evgeny/Projects/internship/src/common'
make[6]: Nothing to be done for 'all'.
make[6]: Leaving directory '/home/evgeny/Projects/internship/src/common'
make[5]: Leaving directory '/home/evgeny/Projects/internship/src/interfaces/libpq'
make -C ../../../../src/interfaces/ecpg/pgtypeslib all
make[5]: Entering directory '/home/evgeny/Projects/internship/src/interfaces/ecpg/pgtypeslib'
make -C ../../../../src/port all
make[6]: Entering directory '/home/evgeny/Projects/internship/src/port'
make[6]: Nothing to be done for 'all'.
make[6]: Leaving directory '/home/evgeny/Projects/internship/src/port'
make -C ../../../../src/common all
make

make[4]: Entering directory '/home/evgeny/Projects/internship/src/port'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/port'
make -C ../../../src/common all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/common'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/common'
make -C ../../../src/fe_utils all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/fe_utils'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/fe_utils'
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/bin/pg_amcheck'
make -C pg_archivecleanup all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/bin/pg_archivecleanup'
make -C ../../../src/port all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/port'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/e

...

gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -O2 -fPIC -shared -o pg_surgery.so  heap_surgery.o -L../../src/port -L../../src/common    -Wl,--as-needed  
make[2]: Leaving directory '/home/evgeny/Projects/internship/contrib/pg_surgery'
make -C pg_trgm all
make[2]: Entering directory '/home/evgeny/Projects/internship/contrib/pg_trgm'
gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -O2 -fPIC -I. -I/home/evgeny/Projects/internship/src/contrib/pg_trgm -I../../src/include -I/home/evgeny/Projects

gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -O2 -fPIC -shared -o pg_walinspect.so  pg_walinspect.o -L../../src/port -L../../src/common    -Wl,--as-needed  
make[2]: Leaving directory '/home/evgeny/Projects/internship/contrib/pg_walinspect'
make -C postgres_fdw all
make[2]: Entering directory '/home/evgeny/Projects/internship/contrib/postgres_fdw'
make -C ../../src/interfaces/libpq all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/interfaces/libpq'
make -C ../../../src/port all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/port'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/port'
make -C ../../../src/common all
make[4]: Entering directory 

gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -O2 -fPIC tcn.o -L../../src/port -L../../src/common   -Wl,--as-needed  -shared -o tcn.so
make[2]: Leaving directory '/home/evgeny/Projects/internship/contrib/tcn'
make -C test_decoding all
make[2]: Entering directory '/home/evgeny/Projects/internship/contrib/test_decoding'
gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -O2 -fPIC -I. -I/home/evgeny/Projects/internship/src/contrib/test_decoding -I../../src/include -I/home/evgeny/Projects/interns

gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -O2 -fPIC -I. -I/home/evgeny/Projects/internship/src/contrib/pgcrypto -I../../src/include -I/home/evgeny/Projects/internship/src/src/include  -D_GNU_SOURCE   -c -o mbuf.o /home/evgeny/Projects/internship/src/contrib/pgcrypto/mbuf.c
gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -O2 -fPIC -I. -I/home/evgeny/Projects/internship/src/contrib/pgcrypto -I../../src/include -I/home/evgeny/Projects/internship/src/src/include  -D_GNU_SOURCE   -c -o open

gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -O2 -fPIC -I. -I/home/evgeny/Projects/internship/src/contrib/pgcrypto -I../../src/include -I/home/evgeny/Projects/internship/src/src/include  -D_GNU_SOURCE   -c -o pgp.o /home/evgeny/Projects/internship/src/contrib/pgcrypto/pgp.c
gcc -Wall -Wmissing-prototypes -Wpointer-arith -Wdeclaration-after-statement -Werror=vla -Wendif-labels -Wmissing-format-attribute -Wimplicit-fallthrough=3 -Wcast-function-type -Wformat-security -fno-strict-aliasing -fwrapv -fexcess-precision=standard -Wno-format-truncation -Wno-stringop-truncation -O2 -fPIC -I. -I/home/evgeny/Projects/internship/src/contrib/pgcrypto -I../../src/include -I/home/evgeny/Projects/internship/src/src/include  -D_GNU_SOURCE   -c -o px-cry

In [46]:
!make install-world

make -C ./src/backend generated-headers
make[1]: Entering directory '/home/evgeny/Projects/internship/src/backend'
make -C catalog distprep generated-header-symlinks
make[2]: Entering directory '/home/evgeny/Projects/internship/src/backend/catalog'
make[2]: Nothing to be done for 'distprep'.
make[2]: Nothing to be done for 'generated-header-symlinks'.
make[2]: Leaving directory '/home/evgeny/Projects/internship/src/backend/catalog'
make -C utils distprep generated-header-symlinks
make[2]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils'
make[2]: Nothing to be done for 'distprep'.
make[2]: Nothing to be done for 'generated-header-symlinks'.
make[2]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils'
make[1]: Leaving directory '/home/evgeny/Projects/internship/src/backend'
make -C doc install
make[1]: Entering directory '/home/evgeny/Projects/internship/doc'
make -C src install
make[2]: Entering directory '/home/evgeny/Projects/internship/doc/sr

make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/parser'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/parser'
make -C commands all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/commands'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/commands'
make -C executor all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/executor'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/executor'
make -C foreign all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/foreign'
make[3]: Nothing to be done for 'all'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/foreign'
make -C lib all
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/lib'
make[3]: Nothing to be do

make -C cache all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils/cache'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils/cache'
make -C error all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils/error'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils/error'
make -C fmgr all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils/fmgr'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils/fmgr'
make -C hash all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils/hash'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils/hash'
make -C init all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/bac

make -C foreign install
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/foreign'
make[3]: Nothing to be done for 'install'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/foreign'
make -C lib install
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/lib'
make[3]: Nothing to be done for 'install'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/lib'
make -C libpq install
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/libpq'
make[3]: Nothing to be done for 'install'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/libpq'
make -C main install
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/main'
make[3]: Nothing to be done for 'install'.
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/main'
make -C nodes install
make[3]: Entering directory '/home/evgeny/Projects/internship/src/backend/nod

make -C time install
make[4]: Entering directory '/home/evgeny/Projects/internship/src/backend/utils/time'
make[4]: Nothing to be done for 'install'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils/time'
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/backend/utils'
make -C ../../src/timezone install
make[3]: Entering directory '/home/evgeny/Projects/internship/src/timezone'
make -C ../../src/port all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/port'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/port'
make -C ../../src/common all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/common'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/common'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/share/postgresql'
./zic -d '/home/evgeny/Projects/internship/.local/share/postgresql/timezo

...

make[1]: Entering directory '/home/evgeny/Projects/internship/contrib'
make -C adminpack install
make[2]: Entering directory '/home/evgeny/Projects/internship/contrib/adminpack'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/lib/postgresql'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/share/postgresql/extension'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/share/postgresql/extension'
/usr/bin/install -c -m 755  adminpack.so '/home/evgeny/Projects/internship/.local/lib/postgresql/adminpack.so'
/usr/bin/install -c -m 644 /home/evgeny/Projects/internship/src/contrib/adminpack/adminpack.control '/home/evgeny/Projects/internship/.local/share/postgresql/extension/'
/usr/bin/install -c -m 644 /home/evgeny/Projects/internship/src/contrib/adminpack/adminpack--1.0.sql /home/evgeny/Projects/internship/src/contrib/adminpack/adminpack--1.0--1.1.sql /home/evgeny/Projects/internship/src/contrib/adminpack/adminpack--1.1--2.0.sql /home/evgeny/Projects/internship/

make[3]: Entering directory '/home/evgeny/Projects/internship/src/interfaces/libpq'
make -C ../../../src/port all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/port'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/port'
make -C ../../../src/common all
make[4]: Entering directory '/home/evgeny/Projects/internship/src/common'
make[4]: Nothing to be done for 'all'.
make[4]: Leaving directory '/home/evgeny/Projects/internship/src/common'
make[3]: Leaving directory '/home/evgeny/Projects/internship/src/interfaces/libpq'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/lib/postgresql'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/share/postgresql/extension'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/share/postgresql/extension'
/usr/bin/install -c -m 755  dblink.so '/home/evgeny/Projects/internship/.local/lib/postgresql/dblink.so'
/usr/bin/install -c -m 644 /home/evgeny/Projects/

make[2]: Entering directory '/home/evgeny/Projects/internship/contrib/lo'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/share/postgresql/extension'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/share/postgresql/extension'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/lib/postgresql'
/usr/bin/install -c -m 644 /home/evgeny/Projects/internship/src/contrib/lo/lo.control '/home/evgeny/Projects/internship/.local/share/postgresql/extension/'
/usr/bin/install -c -m 644 /home/evgeny/Projects/internship/src/contrib/lo/lo--1.1.sql /home/evgeny/Projects/internship/src/contrib/lo/lo--1.0--1.1.sql  '/home/evgeny/Projects/internship/.local/share/postgresql/extension/'
/usr/bin/install -c -m 755  lo.so '/home/evgeny/Projects/internship/.local/lib/postgresql/'
make[2]: Leaving directory '/home/evgeny/Projects/internship/contrib/lo'
make -C ltree install
make[2]: Entering directory '/home/evgeny/Projects/internship/contrib/ltree'
/usr/bin/mkdir -p '/home/evgeny/Pr

/usr/bin/install -c -m 644 /home/evgeny/Projects/internship/src/contrib/pg_trgm/pg_trgm.control '/home/evgeny/Projects/internship/.local/share/postgresql/extension/'
/usr/bin/install -c -m 644 /home/evgeny/Projects/internship/src/contrib/pg_trgm/pg_trgm--1.5--1.6.sql /home/evgeny/Projects/internship/src/contrib/pg_trgm/pg_trgm--1.4--1.5.sql /home/evgeny/Projects/internship/src/contrib/pg_trgm/pg_trgm--1.3--1.4.sql /home/evgeny/Projects/internship/src/contrib/pg_trgm/pg_trgm--1.3.sql /home/evgeny/Projects/internship/src/contrib/pg_trgm/pg_trgm--1.2--1.3.sql /home/evgeny/Projects/internship/src/contrib/pg_trgm/pg_trgm--1.1--1.2.sql /home/evgeny/Projects/internship/src/contrib/pg_trgm/pg_trgm--1.0--1.1.sql  '/home/evgeny/Projects/internship/.local/share/postgresql/extension/'
make[2]: Leaving directory '/home/evgeny/Projects/internship/contrib/pg_trgm'
make -C pgrowlocks install
make[2]: Entering directory '/home/evgeny/Projects/internship/contrib/pgrowlocks'
/usr/bin/mkdir -p '/home/evge

/usr/bin/install -c -m 644 /home/evgeny/Projects/internship/src/contrib/tcn/tcn--1.0.sql  '/home/evgeny/Projects/internship/.local/share/postgresql/extension/'
/usr/bin/install -c -m 755  tcn.so '/home/evgeny/Projects/internship/.local/lib/postgresql/'
make[2]: Leaving directory '/home/evgeny/Projects/internship/contrib/tcn'
make -C test_decoding install
make[2]: Entering directory '/home/evgeny/Projects/internship/contrib/test_decoding'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/lib/postgresql'
/usr/bin/install -c -m 755  test_decoding.so '/home/evgeny/Projects/internship/.local/lib/postgresql/'
make[2]: Leaving directory '/home/evgeny/Projects/internship/contrib/test_decoding'
make -C tsm_system_rows install
make[2]: Entering directory '/home/evgeny/Projects/internship/contrib/tsm_system_rows'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/lib/postgresql'
/usr/bin/mkdir -p '/home/evgeny/Projects/internship/.local/share/postgresql/extension'
/usr/bin/mkdir 